# Setup and Imports

In [1]:
%%capture
!rm -rf fairgraph
!git clone https://github.com/GMattheisen/fairgraph.git
!pip install -r ./fairgraph/requirements.txt
!pip install -U ./fairgraph

In [2]:
from fairgraph import KGClient
import os
import re
import io
import logging
from datetime import date, datetime
from pprint import pprint
import yaml
from collections import defaultdict
from collections import OrderedDict
import requests
import numpy as np
from neo.io import get_io, Spike2IO, NeoMatlabIO, RawBinarySignalIO
from fairgraph import KGClient
from fairgraph.commons import (Species, Handedness, Sex, Strain, Age, Genotype, QuantitativeValue, 
                                BrainRegion, CellType, QuantitativeValueRange, StimulusType)
from fairgraph.core import (Person, Identifier, Material, Organization, Address, Subject, Protocol, Step, use_namespace)
from fairgraph.experiment import (Slice, BrainSlicingActivity, Device, ElectrophysiologicalStimulation,  
                                ElectrophysiologicalStimulus, BehavioralStimulation, BehavioralStimulus, CellCulture, CulturingActivity, CranialWindow, Craniotomy)
from fairgraph.electrophysiology import (PatchedSlice, ExtracellularElectrodeExperiment, PatchedCell, PatchedCellCollection, Distribution, PatchClampActivity, PatchClampExperiment,
                                         Trace, MultiChannelMultiTrialRecording, Distribution, QualifiedMultiTraceGeneration,
                                        ElectrodePlacementActivity, ElectrodeImplantationActivity, Sensor, ImplantedBrainTissue)
from fairgraph.base import KGQuery, as_list
from fairgraph.minds import Dataset

use_namespace('neuralactivity')

/Users/glynismattheisen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(filename="to_knowledge_graph.log",
                    filemode='a',
                    level=logging.DEBUG)

logger = logging.getLogger("nar")

token = ""
client = KGClient(token, nexus_endpoint='https://nexus-int.humanbrainproject.org/v0')

# direct paths dependent on environment
if 'int' in client.nexus_endpoint:
    Subject._path = '/core/subject/v0.1.2'
    QualifiedMultiTraceGeneration._path = '/electrophysiology/multitracegeneration/v0.2.3'
    MultiChannelMultiTrialRecording._path = '/electrophysiology/multitrace/v0.3.1'
    PatchClampExperiment._path = '/electrophysiology/stimulusexperiment/v0.3.1'
    Device._path = '/electrophysiology/device/v0.1.2'
    Material._path = '/core/material/v0.1.2'
    Step._path = '/core/step/v0.1.1'
    Protocol._path = '/core/protocol/v0.1.6'
    CellCulture._path = '/experiment/cellculture/v0.1.0'
    CulturingActivity._path = '/experiment/culturingactivity/v0.2.0'
    Craniotomy._path = '/optophysiology/craniotomy/v0.4.0'
    BehavioralStimulation._path = '/experiment/behavioralstimulation/v0.1.0'
    ExtracellularElectrodeExperiment._path = '/electrophysiology/stimulusexperiment/v0.3.1'

### ExtracellularElectrodeExperiment

#### Components

Subject<br>
Stimulus<br>
Stimulation<br>
Protocol<br>
ImplantedBrainTissue<br>
ExtracellularElectrodeExperiment<br>
ElectrodeImplantationActivity

# Locate Dataset and Define Contributors

In [4]:
dataset_name = None
minds_dataset = Dataset.by_name(dataset_name, client, resolved=True, api="query")

# map MINDSPerson to Person (for owner, contributors)
owner = [obj.resolve(client) for obj in as_list(minds_dataset.owners)]
contributors = [obj.resolve(client) for obj in as_list(minds_dataset.contributors)]

Dataset(activity=Activity(identifier='2d5a91b6e0e0b4039232f4b9ab9259c8', name='fz-juelich.de.2.0003', ethics_authority=EthicsAuthority(identifier='439e80264767d12900f421b866381f53', name='C2EA 71', id=https://nexus.humanbrainproject.org/v0/data/minds/ethics/authority/v1.0.0/996cafcd-f25e-4ee1-a320-0098447422ad), methods=[Method(identifier='25999a726caae1f9b09ffcece7c48854', name='Extracellular recording', id=https://nexus.humanbrainproject.org/v0/data/minds/experiment/method/v1.0.0/aea856db-b1cd-448f-a5e4-9add4d134ca0), Method(identifier='36dae630e8b1bcc62906c6431c048c1e', name='Single unit activity (SUA)', id=https://nexus.humanbrainproject.org/v0/data/minds/experiment/method/v1.0.0/cdf85ba8-d4f7-4ab7-a9a3-5f1a697f778a), Method(identifier='f23a6f84fa4745dfc6f12832c3d5f777', name='Multi unit activity (MUA)', id=https://nexus.humanbrainproject.org/v0/data/minds/experiment/method/v1.0.0/7a60bc78-b32d-45f7-8391-0e49bf4884af), Method(identifier='41fb895e93910c133c68197707db5860', name='Fie

In [ ]:
for minds_person in contributors:
    print(minds_person)

In [5]:
"""People"""
abc = Person(given_name="", family_name="", email="")
abc.save(client)
deg = Person(given_name="", family_name="", email=None)
deg.save(client)
hij = Person(given_name="", family_name="", email=None)
hij.save(client)

people = [abc, deg, hij]

specimen_group = [obj.resolve(client) for obj in as_list(minds_dataset.specimen_group)]
minds_subjects = []
for sg in specimen_group:
    minds_subjects.extend([obj.resolve(client) for obj in as_list(sg.subjects)])

# Define Subjects, Samples, Experiment

In [ ]:
for minds_subject in minds_subjects:
    print(minds_subject.name)
    print(minds_subject.samples)
    for minds_sample in as_list(minds_subject.samples):
        print(as_list(minds_subject.samples))
        minds_sample = minds_sample.resolve(client)
        print(minds_sample.name)

In [7]:
# import trace_dict from previous run if available
if os.path.exists(f'{minds_dataset.name}_trace_dict.yaml'):
    print("loading trace_dict")
    with open(f'{minds_dataset.name}_trace_dict.yaml') as file:
        trace_dict = yaml.load(file, Loader=yaml.FullLoader)
        print(trace_dict)
else:
    print("no saved trace_dict, building trace_dict")
    trace_dict = {}

L
L.l101210-001_raw-lfp-spikes_ch01
L
L.l101210-001_raw-lfp-spikes_ch02
L
L.l101210-001_raw-lfp-spikes_ch03
L
L.l101210-001_raw-lfp-spikes_ch04
L
L.l101210-001_raw-lfp-spikes_ch05
L
L.l101210-001_raw-lfp-spikes_ch06
L
L.l101210-001_raw-lfp-spikes_ch07
L
L.l101210-001_raw-lfp-spikes_ch08
L
L.l101210-001_raw-lfp-spikes_ch09
L
L.l101210-001_raw-lfp-spikes_ch10
L
L.l101210-001_raw-lfp-spikes_ch11
L
L.l101210-001_raw-lfp-spikes_ch12
L
L.l101210-001_raw-lfp-spikes_ch13
L
L.l101210-001_raw-lfp-spikes_ch14
L
L.l101210-001_raw-lfp-spikes_ch15
L
L.l101210-001_raw-lfp-spikes_ch16
L
L.l101210-001_raw-lfp-spikes_ch17
L
L.l101210-001_raw-lfp-spikes_ch18
L
L.l101210-001_raw-lfp-spikes_ch19
L
L.l101210-001_raw-lfp-spikes_ch20
L
L.l101210-001_raw-lfp-spikes_ch21
L
L.l101210-001_raw-lfp-spikes_ch22
L
L.l101210-001_raw-lfp-spikes_ch23
L
L.l101210-001_raw-lfp-spikes_ch24
L
L.l101210-001_raw-lfp-spikes_ch25
L
L.l101210-001_raw-lfp-spikes_ch26
L
L.l101210-001_raw-lfp-spikes_ch27
L
L.l101210-001_raw-lfp-spik

In [8]:
# map MINDSSubject to Subject
for minds_subject in minds_subjects:
    print(minds_subject.name)

    """Subject"""
    species = minds_subject.species.resolve(client).name
    strain = minds_subject.strains
    genotype = minds_subject.genotype
    sex = minds_subject.sex.resolve(client).name
    if sex == "Unknown" or sex == '<Unknown>':
        sex = None
    else:
        sex=Sex(sex)
    handedness = None
    age_units = "days"
    age_value = minds_subject.age
    if "-" in age_value:
        min, max = age_value.split("-")
        age_value = QuantitativeValueRange(int(min), int(max), age_units)
    else:
        age_value = QuantitativeValue(int(age_value), age_units)    
    age_category = "Post-natal"
    death_date = None # date type
    group = None # 'control group' v 'treatment group'

    subject = Subject(
      minds_subject.name,
      species=Species(species),
      strain=Strain(strain),
      genotype=Genotype(genotype),
      sex = sex,
      age=Age(age_value, age_category),
      death_date=None
      )
    print(subject)
    subject.save(client)
    assert subject.id is not None
        
    implanted_brain_tissues = []
    for minds_sample in as_list(minds_subject.samples):
        minds_sample = minds_sample.resolve(client)

        parcellation_regions = [region for region in minds_sample.parcellation_region]
        brain_regions = [BrainRegion("striatum")]  

        implanted_brain_tissue = ImplantedBrainTissue(
            name=f'Tissue from {minds_sample.name}',
            subject=subject
        )
        implanted_brain_tissue.save(client)
        implanted_brain_tissues.append(implanted_brain_tissue)
        
        step_one = Step(
            name = "Step one for protocol",
            previous_step_name = None,
            sequence_number = 2,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            description = "",
            materials - Material(), # or list
            author = Person() # or list
            )
        step_one.save(client)
            
        step_two = Step(
            name = "Step two for protocol",
            previous_step_name = "Step one for protocl",
            sequence_number = 2,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            description = "",
            materials - Material(), # or list
            author = Person() # or list
            )
        step_two.save(client)

        steps = [step_one, step_two]
        
        protocol = Protocol(
            name = "Protcol for ",
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            number_of_steps = 0,
            steps = steps, 
            materials - Material(), # or list
            author = Person(), # or list
            date_published = # date type
            )
        protocol.save(client)

        behavioral_stimulus = BehavioralStimulus(
            name=f"Stimulus for {minds_sample.name}",
            description = "",
            distribution = None
            )
        behavioral_stimulus.save(client)

        behavioral_stimulation = BehavioralStimulation(
            name=f"Stimulation for {minds_sample.name}",
            stimulus = behavioral_stimulus,
            protocol= protocol,
            citation = None,
            code = None,
            license = None
            )
        behavioral_stimulation.save(client)
        
        expt = ExtracellularElectrodeExperiment(
            name = f"LFP recording in {minds_sample.name}",
            recorded_cell =implanted_brain_tissue,
            stimulation = [stimulation],
            traces=[])
        expt.save(client)
        
        file_dir = "hbp-d00000/"

        for item in os.listdir(file_dir):
            filename=item
            file_url=file_dir + filename
            print("file_url", file_url)
            file_location = "https://object.cscs.ch/v1/AUTH_0000000000000000000/" + file_url
            print("file_location", file_location)
        
            io = RawBinarySignalIO(filename = file_url)
            data = io.read_block()
            n_segments = len(data.segments)
            sigs = data.segments[0].analogsignals
            channel_names = [sig.name for sig in sigs]
            units = [sig.units.dimensionality.string for sig in sigs]
            time_step = QuantitativeValue(**{
                                "unit_text": "ms",
                                "value": float(sigs[0].sampling_period.rescale("ms"))
                              })

        file_dir = "hbp-d00000/"

        for item in os.listdir(file_dir):
            filename=item
            file_url=file_dir + filename
            print("file_url", file_url)
            file_location = "https://object.cscs.ch/v1/AUTH_0000000000000000000/" + file_url
            print("file_location", file_location)
        
            io = RawBinarySignalIO(filename = file_url)
            data = io.read_block()
            n_segments = len(data.segments)
            sigs = data.segments[0].analogsignals
            channel_names = [sig.name for sig in sigs]
            units = [sig.units.dimensionality.string for sig in sigs]
            time_step = QuantitativeValue(**{
                                "unit_text": "ms",
                                "value": float(sigs[0].sampling_period.rescale("ms"))
                              })
            
            """Trace Generation"""
            if bool(trace_dict.get(minds_subject.name)):
            # building from YAML
                print("trace_dict exists")
                trace_generation = QualifiedMultiTraceGeneration(
                    f"Metadata for extracellular recording in {minds_subject.name}",
                    stimulus_experiment=expt,
                    sweeps=trace_dict[minds_subject.name]['n_segments'],
                    channel_type = None,
                    holding_potential = QuantitativeValue(int(value), "units"),
                    sampling_frequency = QuantitativeValue(int(value), "units"),
                    power_line_frequency = None
                    )
                trace_generation.save(client)
                print(trace_generation)

                trace = MultiChannelMultiTrialRecording(
                    f"Traces recorded in {minds_subject.name}",
                    data_location=Distribution(location = file_location,
                      content_type="application/octet-stream"),
                    generated_by=expt,
                    generation_metadata=trace_generation,
                    channel_names=trace_dict[minds_subject.name]['channel_names'],
                    data_unit=trace_dict[minds_subject.name]['data_unit'],
                    time_step=trace_dict[minds_subject.name]['time_step'],
                    channel_type = None,
                    part_of=minds_dataset
                    )
                print(trace)
                trace.save(client)
                
            else:
                print("trace_dict does not exist, building trace dict")
                trace_dict[minds_subject.name] = {}
                trace_dict[minds_subject.name]['channel_names'] = channel_names
                trace_dict[minds_subject.name]['data_unit'] = units
                trace_dict[minds_subject.name]['time_step'] = time_step
                trace_dict[minds_subject.name]['n_segments'] = n_segments

                trace_generation = QualifiedMultiTraceGeneration(
                    f"Metadata for extracellular recording in {minds_subject.name}",
                    stimulus_experiment = expt,
                    sweeps = n_segments,
                    channel_type = None,
                    holding_potential = QuantitativeValue(holding_potential_value, holding_potential_unit),
                    sampling_frequency = None, 
                    power_line_frequency = None
                    )
                trace_generation.save(client)
                print(trace_generation)

                trace = MultiChannelMultiTrialRecording(
                    f"Traces recorded in {minds_subject.name}",
                    data_location=Distribution(location = file_location,
                      content_type="application/octet-stream"),
                    generated_by=expt,
                    generation_metadata=trace_generation,
                    channel_names=channel_names,
                    data_unit=units,
                    time_step=time_step,
                    channel_type = None,
                    part_of=minds_dataset
                    )
                print(trace)
                trace.save(client)

    
    electrode = ElectrodeImplantationActivity(
        name = f"Electrode implantation activity for {minds_sample.name}",
        subject = subject,
        brain_location = brain_regions,
        implanted_brain_tissues = implanted_brain_tissues,
        device=device,
        protocol = Protocol(),
        cranial_window = cranial_window,
        anesthesia = "",
        start_time =, # datetime
        end_time = , # datetime
        people=people)
    electrode.save(client)
 

L.l101210-001_raw-lfp-spikes_ch01
FILE EXTENSION l101210-001_raw-lfp-spikes_ch01
MINDS L.l101210-001_raw-lfp-spikes_ch01
l101210-001_raw-lfp-spikes_ch01.mat
SAMPLE NAME L.l101210-001_raw-lfp-spikes_ch01
FILE URL ext-d000015_multielectrode_grasp_pub/L/l101210-001_raw-lfp-spikes_ch01.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in L.l101210-001_raw-lfp-spikes_ch01', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in L.l101210-001_raw-lfp-spikes_ch01', stimulation=[BehavioralStimulation(name='Stimulation for L', behavioral_stimulus=BehavioralStimulus(name='Stimulus for L', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/787eed1f-c401-47c1-ac49-7d0b0c93a867), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralact

FILE EXTENSION l101210-001_raw-lfp-spikes_ch03
MINDS L.l101210-001_raw-lfp-spikes_ch03
l101210-001_raw-lfp-spikes_ch03.mat
SAMPLE NAME L.l101210-001_raw-lfp-spikes_ch03
FILE URL ext-d000015_multielectrode_grasp_pub/L/l101210-001_raw-lfp-spikes_ch03.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in L.l101210-001_raw-lfp-spikes_ch03', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in L.l101210-001_raw-lfp-spikes_ch03', stimulation=[BehavioralStimulation(name='Stimulation for L', behavioral_stimulus=BehavioralStimulus(name='Stimulus for L', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/787eed1f-c401-47c1-ac49-7d0b0c93a867), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION l101210-001_raw-lfp-spikes_ch05
MINDS L.l101210-001_raw-lfp-spikes_ch05
l101210-001_raw-lfp-spikes_ch05.mat
SAMPLE NAME L.l101210-001_raw-lfp-spikes_ch05
FILE URL ext-d000015_multielectrode_grasp_pub/L/l101210-001_raw-lfp-spikes_ch05.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in L.l101210-001_raw-lfp-spikes_ch05', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in L.l101210-001_raw-lfp-spikes_ch05', stimulation=[BehavioralStimulation(name='Stimulation for L', behavioral_stimulus=BehavioralStimulus(name='Stimulus for L', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/787eed1f-c401-47c1-ac49-7d0b0c93a867), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION l101210-001_raw-lfp-spikes_ch07
MINDS L.l101210-001_raw-lfp-spikes_ch07
l101210-001_raw-lfp-spikes_ch07.mat
SAMPLE NAME L.l101210-001_raw-lfp-spikes_ch07
FILE URL ext-d000015_multielectrode_grasp_pub/L/l101210-001_raw-lfp-spikes_ch07.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in L.l101210-001_raw-lfp-spikes_ch07', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in L.l101210-001_raw-lfp-spikes_ch07', stimulation=[BehavioralStimulation(name='Stimulation for L', behavioral_stimulus=BehavioralStimulus(name='Stimulus for L', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/787eed1f-c401-47c1-ac49-7d0b0c93a867), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION l101210-001_raw-lfp-spikes_ch09
MINDS L.l101210-001_raw-lfp-spikes_ch09
l101210-001_raw-lfp-spikes_ch09.mat
SAMPLE NAME L.l101210-001_raw-lfp-spikes_ch09
FILE URL ext-d000015_multielectrode_grasp_pub/L/l101210-001_raw-lfp-spikes_ch09.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in L.l101210-001_raw-lfp-spikes_ch09', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in L.l101210-001_raw-lfp-spikes_ch09', stimulation=[BehavioralStimulation(name='Stimulation for L', behavioral_stimulus=BehavioralStimulus(name='Stimulus for L', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/787eed1f-c401-47c1-ac49-7d0b0c93a867), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

...

FILE EXTENSION i140703-001_raw-lfp-spikes_ch91
MINDS N.i140703-001_raw-lfp-spikes_ch91
i140703-001_raw-lfp-spikes_ch91.mat
SAMPLE NAME N.i140703-001_raw-lfp-spikes_ch91
FILE URL ext-d000015_multielectrode_grasp_pub/N/i140703-001_raw-lfp-spikes_ch91.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in N.i140703-001_raw-lfp-spikes_ch91', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in N.i140703-001_raw-lfp-spikes_ch91', stimulation=[BehavioralStimulation(name='Stimulation for N', behavioral_stimulus=BehavioralStimulus(name='Stimulus for N', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/2f26c6cd-0432-4f72-a855-e290f4b681ee), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION i140703-001_raw-lfp-spikes_ch93
MINDS N.i140703-001_raw-lfp-spikes_ch93
i140703-001_raw-lfp-spikes_ch93.mat
SAMPLE NAME N.i140703-001_raw-lfp-spikes_ch93
FILE URL ext-d000015_multielectrode_grasp_pub/N/i140703-001_raw-lfp-spikes_ch93.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in N.i140703-001_raw-lfp-spikes_ch93', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in N.i140703-001_raw-lfp-spikes_ch93', stimulation=[BehavioralStimulation(name='Stimulation for N', behavioral_stimulus=BehavioralStimulus(name='Stimulus for N', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/2f26c6cd-0432-4f72-a855-e290f4b681ee), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION i140703-001_raw-lfp-spikes_ch95
MINDS N.i140703-001_raw-lfp-spikes_ch95
i140703-001_raw-lfp-spikes_ch95.mat
SAMPLE NAME N.i140703-001_raw-lfp-spikes_ch95
FILE URL ext-d000015_multielectrode_grasp_pub/N/i140703-001_raw-lfp-spikes_ch95.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in N.i140703-001_raw-lfp-spikes_ch95', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in N.i140703-001_raw-lfp-spikes_ch95', stimulation=[BehavioralStimulation(name='Stimulation for N', behavioral_stimulus=BehavioralStimulus(name='Stimulus for N', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/2f26c6cd-0432-4f72-a855-e290f4b681ee), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-18

FILE EXTENSION i140703-001_lfp-spikes
MINDS N.i140703-001_lfp-spikes
i140703-001_lfp-spikes.mat
SAMPLE NAME N.i140703-001_lfp-spikes
FILE URL ext-d000015_multielectrode_grasp_pub/N/i140703-001_lfp-spikes.mat
QualifiedMultiTraceGeneration(name='Metadata for field potentials recorded in N.i140703-001_lfp-spikes', stimulus_experiment=ExtracellularElectrodeExperiment(name='LFP recording in N.i140703-001_lfp-spikes', stimulation=[BehavioralStimulation(name='Stimulation for N', behavioral_stimulus=BehavioralStimulus(name='Stimulus for N', description='TwoCues', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/experiment/behavioralstimulus/v0.1.0/2f26c6cd-0432-4f72-a855-e290f4b681ee), protocol=Protocol(name='TwoCues', steps=[Step(name='TS_ON', sequence_number=1, description='Time when trial start event is internally activated.', id=https://nexus.humanbrainproject.org/v0/data/neuralactivity/core/step/v0.1.0/b8d244d4-1828-4761-9aa9-2b188ea8ccc4), Step(name='TS_OFF', previous_step_n

FILE EXTENSION nev
N.i140703-001.ns2
FILE EXTENSION ns2
N.i140703-001.ns6
FILE EXTENSION ns6
N.i140703-001-03.nev
FILE EXTENSION nev


In [9]:
with open(f'{minds_dataset.name}_trace_dict_mat.yaml', 'w') as file: #save trace dict
    documents = yaml.dump(trace_dict_mat, file)

In [15]:
print(f"Updated on: {datetime.now(tz=None)} on {client.nexus_endpoint}")

Updated on: 2020-06-24 16:33:01.323508 on https://nexus.humanbrainproject.org/v0
